# Assessing the representativity of the UK Parliament following the 2017 General Election

Or: how Theresa May came into power with less than a third of the UK casting a vote for her party.

TL-DR: FPTP + abstention

## Background and definitions

The UK (as many western democracies) is a representative democracy, meaning it elects people to represent them and make decisions.

The basic idea is as follow: the UK is geographically divided into 650 **constituencies** of roughly the same number of voters, each of which holds a mini-election for one **seat** in the House of Commons.

These mini-elections are done using what is called a **First-Past the Post** (FPTP) system, meaning all voters cast one vote for one of the candidate, and the candidate with the more votes wins the seat. This system has the advantage of being easy to understand and put in place, but also has several drawbacks. For example, a candidate does not need 50% support to win, just having the highest score (votes might be split e.g. 45-40-15).

The population legally allowed to cast a vote is called the **electorate**. The number of **valid votes** (votes which are counted to determine seats) is however smaller than the number of people in the electorate, as voting is not mandatory and some votes might be spoiled (i.e. made unvalid for a variety of reasons). The ratio of valid votes against the electorate is called the **turnout**.

After the election, the **government** is elected based on a majority of seats in the House of Commons. The government can then propose all sort of ideas to improve the country, in the form of laws that are voted in the House of Commons by their majority.

To simplify the whole process, candidates are organized into **parties**, which publish a political **manifesto** prior to the election. When you are deciding which candidate to vote for, you can refer to the manifesto to know what are the laws they are planning to pass if they take part in a government.

There are many additional subtleties to consider (electoral pacts, majority or coalitions governments, the official opposition, party leaders, the Monarch's role, etc.) but we will ignore them here.

## 2017 Result

See https://www.bbc.com/news/election/2017/results

Turnout was 68.7%.

Following the election, a government was formed by the *Conservative* party, with support from the *Democratic Unionist Party*.

## Problem definition

We will look at how well the parliament, and the government, actually represent the electorate, using two metrics:
1) how many eligible voters have voted for an actual member of the House of Commons
2) how many eligible voters have actually voted for the government's manifesto.

Some hypotheses:
- We assume voters vote for a party, not a candidate
- We assume voters vote for their preferred choice
- We assume non-voters were not satisfied with any of the choices

About the 2nd hypothesis, we know that it is not exactly the case: as a consequence of FPTP, some people will vote "tactically" for a less-liked but better-placed candidate. This means our analysis will be optimistic relating to the representativity of Parliament.

## Data

The data is from the Electoral Commission website and can be found here : https://www.electoralcommission.org.uk/who-we-are-and-what-we-do/elections-and-referendums/past-elections-and-referendums/uk-general-elections/results-and-turnout-2017-uk-general-election

## Analysis

Before we do anything, let's import some libraries.

In [3]:
import csv

Let's load some administrative data about each constituency

In [7]:
constituencies_info = {}

with open("2017-UKPGE-Electoral-Data - Administrative data.csv", encoding='utf8') as f:
    # Note: there's a typo in the CSV file
    # Party Identifer => Party Identifier
    reader = csv.DictReader(f)

    for row in reader:
        ons = row["ONS Code"]
        name = row["Constituency"]
        electorate_nb = row["Electorate "]  # extra space because why not
        valid_vote_nb = row["Total number of valid votes counted"]

        assert ons not in constituencies_info

        constituencies_info[ons] = {
            'name': name,
            'electorate_nb': int(electorate_nb.replace(',', '')),
            'valid_vote_nb': int(valid_vote_nb.replace(',', ''))
        }

print("Loaded administrative data for", len(constituencies_info.keys()), "constituencies")

Loaded administrative data for  650 constituencies


Now let's load the election results.

In [9]:
results_by_constituencies = {}

with open("2017-UKPGE-Electoral-Data - Results.csv", encoding='utf8') as f:
    reader = csv.DictReader(f)

    for row in reader:
        ons = row["ONS Code"]
        party = row["Party Identifer"]  # typo in the data because why not
        result = int(row["Valid votes"])
        if ons not in results_by_constituencies:
            results_by_constituencies[ons] = {}
        # if party.startswith("Independent"):
        #    party += row["Surname"]
        results_by_constituencies[ons][party] = result

print("Loaded results for ", len(results_by_constituencies.keys()), "constituencies")


Loaded results for  650 constituencies


We can check that we reproduce the election results:

In [10]:
seats = {}

for ons in results_by_constituencies.keys():
    winner = max(results_by_constituencies[ons], key=results_by_constituencies[ons].get)
    if winner not in seats:
        seats[winner] = 0
    seats[winner] += 1

print("== Party seats ==")
for party in seats.keys():
    print("\t", party, seats[party])

== Party seats ==
	 Conservative 317
	 Labour 262
	 Liberal Democrats 12
	 Green Party 1
	 Speaker 1
	 DUP 10
	 Sinn Féin 7
	 Independent 1
	 SNP 35
	 Plaid Cymru 4


There are indeed the results reported by the BBC : https://www.bbc.com/news/election/2017/results

(Note that the BBC counts the Speaker as a Conservative)

Let's also calculate the total number of voters:

In [12]:
electorate_total = 0

for ons in results_by_constituencies.keys():
    electorate = constituencies_info[ons]["electorate_nb"]
    electorate_total += electorate

print("There were", electorate_total, "voters in the election")

There were 46835433 voters in the election


Let's look at our first point: how many people have voted for the candidate which ended up winning their seat?

We call them "happy voters".

In [13]:
happy_total = 0

for ons in results_by_constituencies.keys():
    winner = max(results_by_constituencies[ons], key=results_by_constituencies[ons].get)

    nb_of_happy_voters = results_by_constituencies[ons][winner]
    happy_total += nb_of_happy_voters

print("There were", happy_total, "happy voters in the election")
print("These represent", 100 * round(happy_total / electorate_total, 2), "% of the electorate.")

There were 17990241 happy voters in the election
These represent 38.0 % of the electorate.


And now let's look at the second point: how many people have voted for parties which eventually became part of the government?

We call them "majority voters".

In [20]:
majority_total = 0

for ons in results_by_constituencies.keys():
    nb_of_majority_voters = 0
    if "Conservative" in results_by_constituencies[ons]:
        nb_of_majority_voters += results_by_constituencies[ons]["Conservative"]
    if "DUP" in results_by_constituencies[ons]:
        nb_of_majority_voters += results_by_constituencies[ons]["DUP"]
    majority_total += nb_of_majority_voters

print("There were", majority_total, "majority voters in the election")
print("These represent", 100 * round(majority_total / electorate_total, 2), "% of the electorate.")

There were 13929000 majority voters in the election
These represent 30.0 % of the electorate.


Interesting note here: most media chose to present the "vote share" of each party not against the electorate, but against the valid votes. This gives a boosted number of vote share of 43% for the government.

## Conclusion


We now have the answer for both our questions:

1) Which part of the electorate voted for a winning candidate: 38% (Parliament representativity)

2) Which part of the electorate voted for a Party which ended up in government: 30% (government representativity)

## About the author

I'm just a bored guy in Belgium who wanted to try Jupyter notebook.

I'm generally left-leaning, would have voted Remain and feels strongly about the ineffectiveness of FPTP and how it can be gamed to have a small part of the electorate in control of everything.